# 4.10 Part 1: finalizing the database for customer segmentation

# This script contains the following:
## -Importing libraries
## -Importing previously prepared data
## -Data Security Check
## -Creating regional segmentation of customers
## -Creating flags for low-activity customers
## -Exporting 2 dataframes based on activity level of customers

## This script contains the following:


In [1]:
# import libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [2]:
# import data path
path = r'C:\Users\Bradley Allen\Desktop\Instacart Basket Analysis'

# 01. Import Data Set

In [3]:
# import data frame
final_df = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_cust(4.9p1).pkl'))

In [5]:
final_df.shape #note that dataframe size was reduced due to memory issues in previous exercise (4.8)

(6480166, 27)

In [6]:
final_df.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,first_order,product_id,add_to_cart_order,product_name,...,order_frequency,last_name,gender,state,age,date_joined,n_dependants,fam_status,income,_merge
0,2539329,1,1,2,8,NaN,True,196,1,Soda,...,NaN,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
1,2539329,1,1,2,8,NaN,True,12427,3,Original Beef Jerky,...,NaN,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
2,2398795,1,2,3,7,15.0,False,10258,2,Pistachios,...,NaN,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
3,473747,1,3,3,12,21.0,False,12427,2,Original Beef Jerky,...,Non-frequent customer,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
4,473747,1,3,3,12,21.0,False,10258,3,Pistachios,...,Non-frequent customer,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both


In [7]:
final_df.tail()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,first_order,product_id,add_to_cart_order,product_name,...,order_frequency,last_name,gender,state,age,date_joined,n_dependants,fam_status,income,_merge
6480161,2129269,206209,8,3,17,22.0,False,17500,6,Cinch Sak Extra Large 33 Gallon Trash Bags,...,Non-frequent customer,Donovan,Female,Iowa,74,9/14/2019,3,married,137969,both
6480162,2266710,206209,10,5,18,29.0,False,41213,5,French Loaf,...,Non-frequent customer,Donovan,Female,Iowa,74,9/14/2019,3,married,137969,both
6480163,626363,206209,12,1,12,18.0,False,6846,1,Diet Pepsi Pack,...,NaN,Donovan,Female,Iowa,74,9/14/2019,3,married,137969,both
6480164,626363,206209,12,1,12,18.0,False,23594,14,Medium Salsa,...,NaN,Donovan,Female,Iowa,74,9/14/2019,3,married,137969,both
6480165,626363,206209,12,1,12,18.0,False,16168,16,Large Organic Omega3 Brown Eggs,...,NaN,Donovan,Female,Iowa,74,9/14/2019,3,married,137969,both


# 02. Data Security Check

### The customers dataframe does contain Personal Identifying Information in the form of last & first names. First name column was dropped during previous cleaning to increase processing speed. Will continue to drop last_name column to preserve PII.

In [8]:
final_df = final_df.drop('last_name', 1)

C:\Users\BRADLE~1\AppData\Local\Temp/ipykernel_18628/491618546.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  final_df = final_df.drop('last_name', 1)


In [9]:
#check drop
final_df.shape #drop successful: went from 27 columns to 26. Chose .shape function instead of .head() function due to DF size

(6480166, 26)

# 03. Regional segmentation

In [10]:
# Flagging Northeast Region in a new colum
final_df.loc[final_df['state'].isin(['Maine', 'New Hampshire', 'Vermont', 'Massachusetts', 'Rhode Island', 'Connecticut', 'New York', 'Pennsylvania', 'New Jersey']), 'region'] = 'Northeast'

In [11]:
# Flagging the Midwest regions
final_df.loc[final_df['state'].isin(['Wisconsin', 'Michigan', 'Illinois', 'Indiana', 'Ohio', 'North Dakota', 'South Dakota', 'Nebraska', 'Kansas', 'Minnesota', 'Iowa', 'Missouri']), 'region'] = 'Midwest'

In [12]:
# Flagging South region
final_df.loc[final_df['state'].isin(['Delaware', 'Maryland', 'District of Columbia', 'Virginia', 'West Virginia', 'North Carolina', 'South Carolina', 'Georgia', 'Florida', 'Kentucky', 'Tennessee', 'Mississippi', 'Alabama', 'Oklahoma', 'Texas', 'Arkansas', 'Louisiana']), 'region'] = 'South'

In [13]:
# flaggint West region
final_df.loc[final_df['state'].isin(['Idaho', 'Montana', 'Wyoming', 'Nevada', 'Utah', 'Colorado', 'Arizona', 'New Mexico', 'Washington', 'Oregon', 'California', 'Hawaii']), 'region'] = 'West'

In [14]:
final_df['region'].value_counts(dropna = False)

South        2157878
West         1529150
Midwest      1520513
Northeast    1143174
NaN           129451
Name: region, dtype: int64

In [15]:
# removing row limit to check data 
pd.options.display.max_rows = None

In [16]:
final_df[['user_id', 'state', 'region']].head(10)

,user_id,state,region
0,1,Alabama,South
1,1,Alabama,South
2,1,Alabama,South
3,1,Alabama,South
4,1,Alabama,South
5,1,Alabama,South
6,1,Alabama,South
7,1,Alabama,South
8,1,Alabama,South
9,1,Alabama,South


In [17]:
final_df[['user_id', 'state', 'region']].tail(10)

,user_id,state,region
6480156,206209,Iowa,Midwest
6480157,206209,Iowa,Midwest
6480158,206209,Iowa,Midwest
6480159,206209,Iowa,Midwest
6480160,206209,Iowa,Midwest
6480161,206209,Iowa,Midwest
6480162,206209,Iowa,Midwest
6480163,206209,Iowa,Midwest
6480164,206209,Iowa,Midwest
6480165,206209,Iowa,Midwest


In [18]:
# Generate crosstab for region & loyalty
crosstab = pd.crosstab(final_df['region'], final_df['loyalty_flag'], dropna = False)

## checking crosstab results

In [19]:
crosstab # keep in mind that this is a reduced sample...

loyalty_flag,Loyal customer,New customer,Regular customer
region,,,
Midwest,471217,301560,747736
Northeast,365261,224563,553350
South,676107,423212,1058559
West,489770,302983,736397


There is a demonstrated difference of spending habits between regions, as shown above. The south accounts for the largest customer base, also having the highest amount of Loyal customers and the highest amount of new customers. 

In [20]:
crosstab.to_clipboard()

# 04. creating an exclusion flag for low-activity customers

In [27]:
final_df.loc[final_df['max_order'] < 5, 'exclusion_flag'] = 'Low-activity Customer'

In [28]:
final_df.loc[final_df['max_order']>= 5, 'exclusion_flag'] = 'Frequent Customer'

In [29]:
final_df[['max_order', 'exclusion_flag']].tail(10)

,max_order,exclusion_flag
6480156,12,Frequent Customer
6480157,12,Frequent Customer
6480158,12,Frequent Customer
6480159,12,Frequent Customer
6480160,12,Frequent Customer
6480161,12,Frequent Customer
6480162,12,Frequent Customer
6480163,12,Frequent Customer
6480164,12,Frequent Customer
6480165,12,Frequent Customer


In [30]:
final_df[['max_order', 'exclusion_flag']].head(15)

,max_order,exclusion_flag
0,10,Frequent Customer
1,10,Frequent Customer
2,10,Frequent Customer
3,10,Frequent Customer
4,10,Frequent Customer
5,10,Frequent Customer
6,10,Frequent Customer
7,10,Frequent Customer
8,10,Frequent Customer
9,10,Frequent Customer


In [32]:
final_df['exclusion_flag'].value_counts(dropna = False)

Frequent Customer        6164429
Low-activity Customer     315737
Name: exclusion_flag, dtype: int64

In [33]:
final_df[['region', 'exclusion_flag']].dtypes

region            object
exclusion_flag    object
dtype: object

In [34]:
# converting the region and exclusion colum to category format
final_df[['region', 'exclusion_flag']] = final_df[['region', 'exclusion_flag']].astype('category')

In [35]:
# new DF only showing low-activity customers
low_act_df = final_df[final_df['exclusion_flag'] == 'Low-activity Customer']

In [36]:
low_act_df.head(30)

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,first_order,product_id,add_to_cart_order,product_name,...,gender,state,age,date_joined,n_dependants,fam_status,income,_merge,region,exclusion_flag
76,3343014,4,1,6,11,NaN,True,35469,3,Enchilada Black Bean Vegetable,...,Male,Arkansas,31,9/21/2017,3,married,45435,both,South,Low-activity Customer
77,94891,4,4,5,13,15.0,False,25146,2,Original Orange Juice,...,Male,Arkansas,31,9/21/2017,3,married,45435,both,South,Low-activity Customer
78,2717275,5,1,3,12,NaN,True,21413,2,Organic Soba,...,Female,California,75,10/8/2018,0,divorced/widowed,115242,both,West,Low-activity Customer
79,2717275,5,1,3,12,NaN,True,31717,7,Organic Cilantro,...,Female,California,75,10/8/2018,0,divorced/widowed,115242,both,West,Low-activity Customer
80,1909121,5,2,0,16,11.0,False,43693,1,Whole Vitamin D Milk,...,Female,California,75,10/8/2018,0,divorced/widowed,115242,both,West,Low-activity Customer
81,1909121,5,2,0,16,11.0,False,21616,5,Organic Baby Arugula,...,Female,California,75,10/8/2018,0,divorced/widowed,115242,both,West,Low-activity Customer
82,2267326,5,3,3,18,10.0,False,26604,1,Organic Blackberries,...,Female,California,75,10/8/2018,0,divorced/widowed,115242,both,West,Low-activity Customer
83,157374,5,4,1,18,19.0,False,3376,8,Organic Whole Kernel Sweet Corn No Salt Added,...,Female,California,75,10/8/2018,0,divorced/widowed,115242,both,West,Low-activity Customer
84,157374,5,4,1,18,19.0,False,8518,9,Organic Red Onion,...,Female,California,75,10/8/2018,0,divorced/widowed,115242,both,West,Low-activity Customer
85,157374,5,4,1,18,19.0,False,26604,10,Organic Blackberries,...,Female,California,75,10/8/2018,0,divorced/widowed,115242,both,West,Low-activity Customer


In [37]:
low_act_df.shape

(315737, 28)

In [38]:
# new DF only containing frequent customers
Insta_final_df = final_df[final_df['exclusion_flag'] == 'Frequent Customer']

In [39]:
Insta_final_df.head(10)

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,first_order,product_id,add_to_cart_order,product_name,...,gender,state,age,date_joined,n_dependants,fam_status,income,_merge,region,exclusion_flag
0,2539329,1,1,2,8,NaN,True,196,1,Soda,...,Female,Alabama,31,2/17/2019,3,married,40423,both,South,Frequent Customer
1,2539329,1,1,2,8,NaN,True,12427,3,Original Beef Jerky,...,Female,Alabama,31,2/17/2019,3,married,40423,both,South,Frequent Customer
2,2398795,1,2,3,7,15.0,False,10258,2,Pistachios,...,Female,Alabama,31,2/17/2019,3,married,40423,both,South,Frequent Customer
3,473747,1,3,3,12,21.0,False,12427,2,Original Beef Jerky,...,Female,Alabama,31,2/17/2019,3,married,40423,both,South,Frequent Customer
4,473747,1,3,3,12,21.0,False,10258,3,Pistachios,...,Female,Alabama,31,2/17/2019,3,married,40423,both,South,Frequent Customer
5,2254736,1,4,4,7,29.0,False,25133,4,Organic String Cheese,...,Female,Alabama,31,2/17/2019,3,married,40423,both,South,Frequent Customer
6,431534,1,5,4,15,28.0,False,196,1,Soda,...,Female,Alabama,31,2/17/2019,3,married,40423,both,South,Frequent Customer
7,431534,1,5,4,15,28.0,False,10258,3,Pistachios,...,Female,Alabama,31,2/17/2019,3,married,40423,both,South,Frequent Customer
8,3367565,1,6,2,7,19.0,False,10258,3,Pistachios,...,Female,Alabama,31,2/17/2019,3,married,40423,both,South,Frequent Customer
9,2550362,1,10,4,8,30.0,False,196,1,Soda,...,Female,Alabama,31,2/17/2019,3,married,40423,both,South,Frequent Customer


In [40]:
Insta_final_df.shape

(6164429, 28)

# Exporting dataframes

In [41]:
low_act_df.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'low_activity_customers.pkl'))

In [42]:
Insta_final_df.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'Instacart_final.pkl'))